<a href="https://colab.research.google.com/github/hegdenishit84/MachineLearning/blob/master/Black_Friday_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
url = 'https://raw.githubusercontent.com/hegdenishit84/MachineLearning/master/train.csv'
df = pd.read_csv(url)

FileNotFoundError: ignored

In [3]:
df.head()

NameError: ignored

In [4]:
df.info()

NameError: ignored

In [0]:
#convert Gender 
df['Gender'] = df['Gender'].map({'M':0, 'F':1})

In [0]:
df.info()

In [0]:
sns.countplot(x='Age', data=df,order=sorted(df['Age'].unique()))

In [0]:
sns.countplot(x='City_Category', data=df, hue='Marital_Status',order=sorted(df['City_Category'].unique()))

In [0]:
plt.figure(figsize=(12,8))
sns.countplot(x='Occupation', data=df,order=sorted(df['Occupation'].unique()))

In [0]:
df[df['Product_Category_1'] == 20].head(2)

In [0]:
df['Product_Category_2'].isnull().groupby(df['Product_Category_1']).mean() * 100

In [0]:
df['Product_Category_3'].isnull().groupby(df['Product_Category_1']).mean() *100

In [5]:
print(sorted(df['Age'].unique()))
print(sorted(df['Stay_In_Current_City_Years'].unique()))
print(sorted(df['City_Category'].unique()))
print(sorted(df['Occupation'].unique()))

NameError: ignored

In [0]:
df['Age'].value_counts()

In [6]:
plt.figure(figsize=(10,8))
sns.boxplot(x="Occupation", y="Purchase", data=df,)

NameError: ignored

<Figure size 720x576 with 0 Axes>

In [7]:
modified_df = pd.get_dummies(df,drop_first=True,columns=['Age','Stay_In_Current_City_Years','City_Category','Occupation'])


NameError: ignored

In [0]:
modified_df.info()

In [0]:
df.info()

In [0]:
def remove_prefix(product_id):
    if(product_id.startswith("P00")):
        return product_id[3:]
    

modified_df['Product_ID'] = modified_df['Product_ID'].apply(lambda x : remove_prefix(x))

In [8]:
modified_df['Product_ID'] = pd.to_numeric(modified_df['Product_ID'])

NameError: ignored

In [0]:
modified_df['Product_ID'].nunique()

In [0]:
modified_df = modified_df.drop(['Product_Category_2', 'Product_Category_3'], axis=1)

In [0]:
modified_df.corr()['Purchase']

In [0]:
#%% Prepare Data
from sklearn.model_selection import train_test_split

# Split-out validation dataset
X = modified_df.drop('Purchase',axis=1).values
y = modified_df['Purchase'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
scaler = StandardScaler()

In [0]:
X_train = scaler.fit_transform(X_train)

In [0]:
X_test = scaler.transform(X_test)

In [0]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
#%% Spot Check Algorithms
models = []
models.append(('LR', LinearRegression()))
models.append(('LASSO', Lasso()))
models.append(('EN', ElasticNet()))

In [0]:
#%% evaluate each model in turn
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=5, shuffle=True, random_state=5)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, \
                                 scoring='neg_mean_squared_error') 
    results.append(cv_results)
    names.append(name)
    
    rmse = np.sqrt(cv_results.mean()*-1)
    msg = "%s: %f (%f)" % (name, rmse, cv_results.std())
    print(msg)
#     print(sorted(cv_results, reverse=True))

In [0]:
ensembles = []
ensembles.append(('AB', AdaBoostRegressor()))
ensembles.append(('GBM', GradientBoostingRegressor()))
ensembles.append(('RF', RandomForestRegressor()))
ensembles.append(('ET', ExtraTreesRegressor()))
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=5, shuffle=True, random_state=5)
    cv_results = cross_val_score(model, X_train, Y_train,cv=kfold, scoring='neg_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    rmse = np.sqrt(cv_results.mean()*-1)
    msg = "%s: %f (%f)" % (name, rmse, cv_results.std())
    print(msg)




In [0]:
#%% Tune scaled GBM
param_grid = dict(n_estimators=np.array([50,100,150,200,250,300,350,400,500,600,700,800]))
                                            
model = GradientBoostingRegressor(random_state=5)
kfold = KFold(n_splits=5, shuffle=True, random_state=5)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=kfold)
grid_result = grid.fit(X_train, Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))